In [25]:
from idxutils import gen_in_network_links, JSONOpen
from tqdm import tqdm
import pandas as pd
import requests
import sys
from _utils import istarmap
import aiohttp
import asyncio
from idxutils import gen_in_network_links
import traceback as tb


In [37]:
async def get_network_files(session, url):
    try:
        async with session.head(url) as r:
            size = r.headers.get("Content-Length")
            return url, size

                    
    except aiohttp.ClientResponseError as e:
        print(e)
        return url, None


    except asyncio.TimeoutError:
        return url, None

    except aiohttp.ClientConnectionError:
        return url, None

    except KeyboardInterrupt:
        import os; os.exit()

    # except UnicodeDecodeError:
    #     tb.print_exc()        
    except Exception as e:
        tb.print_exc()
        print(e)
        return url, None

In [39]:
from tqdm.asyncio import tqdm
  # Pandas
async def main():
    df = pd.read_csv("output.csv")
    df1 = pd.DataFrame(columns=["url", "size"])
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        df1[["url", "size"]] = await tqdm.gather(*(get_network_files(session, url) for url in df["url"]), leave=True, position=0)
        return df1

# File
async def main_file():
    df1 = pd.DataFrame(columns=["url", "size"])
    with open("output2.csv", "r") as f:
        total = len(f.readlines())
        f.seek(0)
        async with aiohttp.ClientSession(raise_for_status = True) as session:
            df1[["url", "size"]] = await tqdm.gather(*(get_network_files(session, url) for url in f), leave=True, position=0)
            return df1

In [40]:
t = await main()

 62%|██████▏   | 62438/100563 [01:11<00:16, 2370.73it/s]

404, message='Not Found', url=URL('https://uhc-tic-mrf.azureedge.net/public-mrf/2023-01-01/2023-01-01_UnitedHealthcare-of-California_INSURER_OPH-%EF%BF%BD-Optum-Physical-Health_OPH-193_in-network-rates.json.gz')
404, message='Not Found', url=URL('https://uhc-tic-mrf.azureedge.net/public-mrf/2023-01-01/2023-01-01_UnitedHealthcare-of-California_INSURER_OPH-%EF%BF%BD-Optum-Physical-Health_OPH-193_in-network-rates.json.gz')


 63%|██████▎   | 63390/100563 [01:11<00:17, 2085.52it/s]

404, message='Not Found', url=URL('https://uhc-tic-mrf.azureedge.net/public-mrf/2023-01-01/2023-01-01_UnitedHealthcare-of-California_INSURER_OPH-%EF%BF%BD-Optum-Physical-Health_OPH-193_in-network-rates.json.gz')
404, message='Not Found', url=URL('https://uhc-tic-mrf.azureedge.net/public-mrf/2023-01-01/2023-01-01_UnitedHealthcare-of-California_INSURER_OPH-%EF%BF%BD-Optum-Physical-Health_OPH-193_in-network-rates.json.gz')


100%|██████████| 100563/100563 [01:28<00:00, 1135.88it/s]


In [23]:
t

,url,size
0,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,621666475
1,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,656852668
2,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,222733
3,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,406373
4,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,2727221
...,...,...
484,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,15757819
485,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,15757819
486,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,15757819
487,https://uhc-tic-mrf.azureedge.net/public-mrf/2...,15757819


In [24]:
t.to_csv("UHC_payers.csv", index=False)